## Step four: Clustering (binning) the latent representation

__The role of clustering in Vamb__

Fundamentally, the process of binning is just clustering sequences based on some of their properties. The purpose of encoding the contigs to a lossy latent representation is to ease the process of clustering because contigs with similar properties are placed close together in latent space, and the latent space is smaller than the input feature space.

With the latent representation conveniently represented by an (n_contigs x n_features) matrix, you could use any clustering algorithm to cluster them (such as the ones in `sklearn.cluster`). In practice though, you have perhaps a million contigs and prior constrains on the diameter, shape and size of the clusters, so non-custom clustering algorithms will probably be slow and inaccurate.

The module `vamb.cluster` implements a simple and fast iterative medoid clustering algorithm. It is well suited for spherical clusters with a maximum size and for many samples. The algorithm is similar, but subtly different from that used in the metagenomic binner Canopy:

    Clustering algorithm used by Vamb:
    (1): Pick random seed observation S
    (2): Define inner_obs(S) = all observations with Pearson distance from S < INNER
    (3): Sample MOVES observations I from inner_obs
    (4): If any mean(inner_obs(i)) < mean(inner_obs(S)) for i in I: Let S be i, go to (2)
         Else: Outer_obs(S) = all observations with Pearson distance from S < OUTER
    (5): Output outer_obs(S) as cluster, remove inner_obs(S) from observations
    (6): If no more observations or MAX_CLUSTERS have been reached: Stop
         Else: Go to (1)

__Determining clustering threshold__

You will notice that this algorithm depends on the parameters `INNER` and `OUTER`. This corresponds to the thresholds for Pearson distance, any contigs within which is considered part of the same bin. Getting this measure right is crucial for the binning to work well. Put them too low, and the bins will be highly fragmented. Too high, and distinct genomes will be binned together.

In order to estimate a good threshold for this distance, we have written the module `threshold`, in where there is the function `getthreshold`:

---

In [1]:
import sys
sys.path.append('/home/jakni/Documents/scripts/')
import vamb

help(vamb.threshold.getthreshold)

Help on function getthreshold in module vamb.threshold:

getthreshold(latent, distfunction, samples, maxsize)
    Estimate the clustering threshold from random sampling.
    
    Inputs:
        latent: An (n_contigs x n_latent) Numpy array
        samples: Number of random contigs to sample [1000]
        maxsize: Discard sample if more than N contigs are within estimated
                 sample threshold [2500]
        
    Output:
        median: Median of estimated clustering threshold
        support: Fraction of contigs with any estimated threshold
        separation: Fraction of contigs well separated



---
How does this work? If you pick a random contig and calculate the distance to all other contigs, the distributions will, roughly speaking follow one of the three distributions, plotted below in blue, orange and green:

---

![title](thresholdcurve.png)

---
This curve is with faked data just for demonstration purposes - in reality, it's a bit more spiky.

Anyway, the basic idea here is that the distances relative to any contig sometimes separate neatly into a small group of close contigs and all the other contigs. The close contigs can then be assumed to be the within-bin and the far contigs the ones outside the bin.

When sampling, we typically see one of three patterns:

* The well-separating, here plotted in __green__. For these samples, the smaller peak of close contigs is clearly separated from the further contigs, and the optimal threshold distance is clearly somewhere in the range 0.1 - 0.15. Formally, we define the distances as well-separated as if the density falls to 0.025 or below between the two peaks. The threshold for this sample is then defined as the distance where the density falls to 0.025.

* The poorly-seperating samples, here plotted in __orange__. You can see a noticable dip in density at about distance = 0.1, so it appears there indeed is a group of close contigs and a group of far contigs, but the separation is less clear. These are the ones where the density falls to <= 80% of the peak density (in this plot, the valley at ~0.1 is < 0.8 * the peak at ~0.6). The threshold for poorly-separating samples is the point where the density falls below 80% of peak density.

* The non-separating samples, here in __blue__. The density at different distances is monotonically increasing, and we cannot find any suitable threshold for these contigs. Formally, we define these as where there are no valleys where the density fall below 80% of the peak value. No threshold is returned for these non-separating samples.

We then have defined two terms:

* The __support__ is the fraction of well-separating and/or poorly-separating samples.

* The __separation__ is the fraction of well-separating samples.

---
The `vamb.threshold.getthreshold` function samples a number of random contigs (1000 by default) and returns the median threshold. Of course, if the support is too low (< 50%) or the separation too low (< 25%), you can't really trust the threshold.

Okay, let's give it a spin

---

In [9]:
# Load the data we created in chapter 3
with open('/home/jakni/Downloads/example/latent.npz', 'rb') as file:
    latent = vamb.vambtools.read_npz(file)

# We need to normalize the array first
vamb.vambtools.zscore(latent, axis=1, inplace=True)

_ = vamb.threshold.getthreshold(latent, distfunction=vamb.cluster._pearson_distances,
                                maxsize=2500, samples=1000)
threshold, support, separation = _

print('Estimated threshold:', round(threshold, 3))
print('Support:', round(support * 100, 1), '%')
print('Separation:', round(separation * 100, 1), '%')

Estimated threshold: 0.041
Support: 44.8 %
Separation: 0.2 %


---
Uh oh, only 0.2 % separation, but more support. So about half the contigs has a little dip in the beginning, and the other half doesn't. This is not very good.

It's not that surprising it acts that badly. I only trained the VAE for 5 epochs, far from enough, so the latent representation is totally unrepresentative of the underlying data.

I'll cheat here and load in a latent representation I made using the `metabat_errorfree` dataset that comes with [the metabat binner](https://bitbucket.org/berkeleylab/metabat/src/master/).

---

In [10]:
# Open gzipped files with the Reader
# This is the latent encoding of the metabat dataset
with open('/home/jakni/Downloads/example/metabat_latent.tsv') as file:
    latent = vamb.vambtools.read_tsv(file)
    
latent.shape

(182388, 40)

---
180,000 contigs with 40 latent neurons each. I've trained this for 200 epochs. This should separate more easily.

---

In [11]:
vamb.vambtools.zscore(latent, axis=1, inplace=True)
_ = vamb.threshold.getthreshold(latent, distfunction=vamb.cluster._pearson_distances,
                                maxsize=2500, samples=1000)
threshold, support, separation = _

print('Estimated threshold:', round(threshold, 3))
print('Support:', round(support * 100, 1), '%')
print('Separation:', round(separation * 100, 1), '%')

Estimated threshold: 0.116
Support: 66.4 %
Separation: 49.6 %


*Much* better. Now with a threshold, we can cluster. In Vamb, we have implemented the algorithm as two distinct functions: 

* `vamb.cluster.cluster`, simply clusters a matrix, and so scales approximately O(n<sup>2</sup>).

* `vamb.cluster.tandemcluster` does some very rough preclustering and then clusters each precluster using `vamb.cluster.cluster`. Each observation is then assigned uniquely to the largest cluster it's a member of. This scales better with number of contigs, but is also significantly less accurate.

You can use the slow-but-accurate with up to one or two million contigs depending on your patience or ~10 million contigs if you're alright with running it for days.

The heavy lifting here is done in Numpy, so it might be worth making sure the BLAS library your Numpy is using is fast. You can check it with `numpy.__config__.show()` and if it says anything other than `NOT AVAILABLE` under the `mkl` or `openblas` entries, you're golden.

---

In [13]:
help(vamb.cluster.cluster)

Help on function cluster in module vamb.cluster:

cluster(matrix, labels=None, inner=None, outer=None, max_steps=15, normalized=False, nsamples=1000, maxsize=2500)
    Iterative medoid cluster generator. Yields (medoid), set(labels) pairs.
    
    Inputs:
        matrix: A (obs x features) Numpy matrix of values
        labels: None or Numpy array with labels for matrix rows [None = ints]
        inner: Optimal medoid search within this distance from medoid [None = auto]
        outer: Radius of clusters extracted from medoid. [None = inner]
        max_steps: Stop searching for optimal medoid after N futile attempts [15]
        normalized: Matrix is already zscore-normalized [False]
        nsamples: Estimate threshold from N samples [1000]
        maxsize: Discard sample if more than N contigs are within threshold [2500]
    
    Output: Generator of (medoid, set(labels_in_cluster)) tuples.



In [17]:
import pickle
import numpy as np

# As written in the help above, labels must be a Numpy array!
# If you pass labels=None, it will use the contig numbers 1 to len(latent)
# as labels.
with open('/home/jakni/Downloads/example/metabat_contignames.npz', 'rb') as file:
    labels = vamb.vambtools.read_npz(file)

# Unlike tandemcluster, which outputs the dictionary directly,
# the output of cluster is a generator
cluster_iterator = vamb.cluster.cluster(latent, labels, threshold)

# Collect the clusters (bins) in a dictionary of sets
clusters = dict()
for medoid, contigs in cluster_iterator:
    clusters[medoid] = contigs

print('Last key:', medoid, '(of type:', type(medoid), ')')
print('Type of values:', type(contigs))
print('First element of value:', next(iter(contigs)), 'of type:', type(next(iter(contigs))))

Last key: gi|345651601|ref|NZ_JH114322.1|_[Bacteroides_dorei_5_1_36_D4_uid55593]_1189469-1207316 (of type: <class 'numpy.str_'> )
Type of values: <class 'set'>
First element of value: gi|345651601|ref|NZ_JH114322.1|_[Bacteroides_dorei_5_1_36_D4_uid55593]_1189469-1207316 of type: <class 'numpy.str_'>


---
In practise, we assume people never want to run the clustering without having determined the threshold first, so if you set `threshold=None` in the `cluster` or `tandemcluster` function, it will run `threshold.getthreshold` on the data first and use whatever threshold was returned.

If the separation or support for this threshold estimation is alarmingly low, it will print a warning to stderr. If the threshold cannot be estimated because the support is so extremely low, it will arbitrarily use 0.08 and print a warning to stderr.

---

## Step five: Postprocessing the clusters

We haven't written any postprocessing modules because how to postprocess really depends on what you're looking for in your data.

One of the greatest weaknesses of Vamb - probably of metagenomic binners in general - is that the bins tend to be highly fragmented. You'll have lots of tiny bins, some of which are legitimate (viruses, plasmids), but most are parts of larger genomes that didn't get binned properly - about 2/3 of the bins here, for example, are 1-contig bins.

We're in the process of developing a tool for annotating, cleaning and merging bins based on phylogenetic analysis of the genes in the bins. That would be extremely helpful, but for now, we'll have to use more crude approaches:

We throw away all bins with less than 250,000 basepairs.

---

In [21]:
# First let's make a contignames: length dict
with open('/home/jakni/Downloads/example/metabat_lengths.npz', 'rb') as file:
    lengths = vamb.vambtools.read_npz(file)

lengthof = dict(zip(labels, lengths))

# Now filter away the small bins
filtered_bins = dict()

for medoid, contigs in clusters.items():
    binsize = sum(lengthof[contig] for contig in contigs)
    
    if binsize >= 250000:
        filtered_bins[medoid] = contigs

print('Number of bins before filtering:', len(clusters))
print('Number of bins after filtering:', len(filtered_bins))

Number of bins before filtering: 24491
Number of bins after filtering: 313


---
Now, let's save the clusters to disk. For this we will use two writer functions:

1) `vamb.cluster.writeclusters`, that writes which clusters contains which contigs to a simple tab-separated file, and

2) `vamb.vambtools.writebins`, that writes FASTA files corresponding to each of the bins to a directory.

We will need to load all the contigs belonging to any bin into memory to use `vamb.vambtools.writebins`. If the contigs in your bins don't fit in memory, sorry, you gotta find another way to make those FASTA bins.

The cluster name when printing either way will be the dictionary key of the bins. Right now, our bins have names like `gi|345651601|ref|NZ_JH114322.1|_[Bacteroides_dorei_5_1_36_D4_uid55593]_1189469-1207316` - not exactly poetic. We'll rename the bins first.

---

In [25]:
# Rename bin keys to something less horrible as a file name
filtered_bins = {'cluster_' + str(i+1): v for i, v in enumerate(filtered_bins.values())}

with open('/home/jakni/Downloads/example/bins.tsv', 'w') as file:
    vamb.cluster.writeclusters(file, filtered_bins)

# Only keep contigs in any filtered bin in memory
allcontigs = set.union(*filtered_bins.values())

with open('/home/jakni/Downloads/example/contigs.fna', 'rb') as file:
    fastadict = vamb.vambtools.loadfasta(file, keep=allcontigs)
    
vamb.vambtools.writebins('/home/jakni/Downloads/example/bins/', filtered_bins, fastadict)